#### books recommender system using clustering | collaborative filtering


In [1]:
# import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline


In [2]:
# loading the data
books = pd.read_csv('data/books_data/books.csv',sep = ';', error_bad_lines=False, encoding="latin-1")
books.head()

b'Skipping line 6452: expected 8 fields, saw 9\nSkipping line 43667: expected 8 fields, saw 10\nSkipping line 51751: expected 8 fields, saw 9\n'
b'Skipping line 92038: expected 8 fields, saw 9\nSkipping line 104319: expected 8 fields, saw 9\nSkipping line 121768: expected 8 fields, saw 9\n'
b'Skipping line 144058: expected 8 fields, saw 9\nSkipping line 150789: expected 8 fields, saw 9\nSkipping line 157128: expected 8 fields, saw 9\nSkipping line 180189: expected 8 fields, saw 9\nSkipping line 185738: expected 8 fields, saw 9\n'
b'Skipping line 209388: expected 8 fields, saw 9\nSkipping line 220626: expected 8 fields, saw 9\nSkipping line 227933: expected 8 fields, saw 11\nSkipping line 228957: expected 8 fields, saw 10\nSkipping line 245933: expected 8 fields, saw 9\nSkipping line 251296: expected 8 fields, saw 9\nSkipping line 259941: expected 8 fields, saw 9\nSkipping line 261529: expected 8 fields, saw 9\n'


,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...
2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton &amp; Company,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...


In [3]:
# shape of the data
books.shape

(271360, 8)

In [4]:
# selecting the columns
books.columns

Index(['ISBN', 'Book-Title', 'Book-Author', 'Year-Of-Publication', 'Publisher',
       'Image-URL-S', 'Image-URL-M', 'Image-URL-L'],
      dtype='object')

In [5]:
books = books[['ISBN', 'Book-Title', 'Book-Author', 'Year-Of-Publication', 'Publisher','Image-URL-L']]
books.head()

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-L
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...
2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial,http://images.amazon.com/images/P/0060973129.0...
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux,http://images.amazon.com/images/P/0374157065.0...
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton &amp; Company,http://images.amazon.com/images/P/0393045218.0...


In [6]:
# changing the column names
books.columns = ['isbn', 'title', 'author', 'year', 'publisher', 'image_url']
books.head()

,isbn,title,author,year,publisher,image_url
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...
2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial,http://images.amazon.com/images/P/0060973129.0...
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux,http://images.amazon.com/images/P/0374157065.0...
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton &amp; Company,http://images.amazon.com/images/P/0393045218.0...


In [7]:
# let's load user information
users = pd.read_csv('data/books_data/users.csv',sep = ';', error_bad_lines=False, encoding="latin-1")
users.head()

,User-ID,Location,Age
0,1,"nyc, new york, usa",NaN
1,2,"stockton, california, usa",18.0
2,3,"moscow, yukon territory, russia",NaN
3,4,"porto, v.n.gaia, portugal",17.0
4,5,"farnborough, hants, united kingdom",NaN


In [8]:
# shape of the data
users.shape

(278858, 3)

In [9]:
# change the column names of user
users.columns = ['user_id', 'location', 'age']
users.head()

,user_id,location,age
0,1,"nyc, new york, usa",NaN
1,2,"stockton, california, usa",18.0
2,3,"moscow, yukon territory, russia",NaN
3,4,"porto, v.n.gaia, portugal",17.0
4,5,"farnborough, hants, united kingdom",NaN


In [10]:
# let's load ratings information
ratings = pd.read_csv('data/books_data/ratings.csv',sep = ';', error_bad_lines=False, encoding="latin-1")
ratings.head()

,User-ID,ISBN,Book-Rating
0,276725,034545104X,0
1,276726,0155061224,5
2,276727,0446520802,0
3,276729,052165615X,3
4,276729,0521795028,6


In [11]:
# shape of the data
ratings.shape

(1149780, 3)

In [12]:
# changing the column names of ratings
ratings.columns = ['user_id', 'isbn', 'rating']
ratings.head()

,user_id,isbn,rating
0,276725,034545104X,0
1,276726,0155061224,5
2,276727,0446520802,0
3,276729,052165615X,3
4,276729,0521795028,6


In [13]:
# finding the ratings of books by each user
ratings['user_id'].value_counts()

11676     13602
198711     7550
153662     6109
98391      5891
35859      5850
          ...  
116180        1
116166        1
116154        1
116137        1
276723        1
Name: user_id, Length: 105283, dtype: int64

You can observe the user 11676 has rated a lot of books. may be he or she is a bibliophile

In [14]:
# selecting the users who rated more than 200 books
counts1 = ratings['user_id'].value_counts()
ratings = ratings[ratings['user_id'].isin(counts1[counts1 >= 200].index)]
ratings.head()

,user_id,isbn,rating
1456,277427,002542730X,10
1457,277427,0026217457,0
1458,277427,003008685X,8
1459,277427,0030615321,0
1460,277427,0060002050,0


In [15]:
# lets find the ratings given by each user
ratings['user_id'].value_counts()

11676     13602
198711     7550
153662     6109
98391      5891
35859      5850
          ...  
225595      200
83671       200
252827      200
36554       200
99955       200
Name: user_id, Length: 905, dtype: int64

 you can observe the data that we only selected the users who rated more than 200 books

In [16]:
# shape of the data
ratings['user_id'].unique().shape

(905,)

 we can observe that there are only 905 unique users who rated more than 200 books

In [17]:
# merging the ratings and books data
combine_book_rating = pd.merge(ratings, books, on='isbn')
combine_book_rating.head()

,user_id,isbn,rating,title,author,year,publisher,image_url
0,277427,002542730X,10,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,http://images.amazon.com/images/P/002542730X.0...
1,3363,002542730X,0,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,http://images.amazon.com/images/P/002542730X.0...
2,11676,002542730X,6,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,http://images.amazon.com/images/P/002542730X.0...
3,12538,002542730X,10,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,http://images.amazon.com/images/P/002542730X.0...
4,13552,002542730X,0,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,http://images.amazon.com/images/P/002542730X.0...


In [18]:
combine_book_rating.shape

(488742, 8)

In [19]:
# finding the count of each book rating
number_of_ratings = combine_book_rating.groupby('title')['rating'].count().reset_index()
number_of_ratings.head()

,title,rating
0,A Light in the Storm: The Civil War Diary of ...,2
1,Always Have Popsicles,1
2,Apple Magic (The Collector's series),1
3,Beyond IBM: Leadership Marketing and Finance ...,1
4,Clifford Visita El Hospital (Clifford El Gran...,1


we are going to select the books which have more than 50 ratings

In [20]:
# change the column name to number of ratings
number_of_ratings.columns = ['title', 'number_of_ratings']
number_of_ratings.head()

,title,number_of_ratings
0,A Light in the Storm: The Civil War Diary of ...,2
1,Always Have Popsicles,1
2,Apple Magic (The Collector's series),1
3,Beyond IBM: Leadership Marketing and Finance ...,1
4,Clifford Visita El Hospital (Clifford El Gran...,1


In [21]:
# merging the number of ratings and combine_book_rating data
final_rating = pd.merge(combine_book_rating, number_of_ratings, on='title')
final_rating.head()

,user_id,isbn,rating,title,author,year,publisher,image_url,number_of_ratings
0,277427,002542730X,10,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,http://images.amazon.com/images/P/002542730X.0...,82
1,3363,002542730X,0,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,http://images.amazon.com/images/P/002542730X.0...,82
2,11676,002542730X,6,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,http://images.amazon.com/images/P/002542730X.0...,82
3,12538,002542730X,10,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,http://images.amazon.com/images/P/002542730X.0...,82
4,13552,002542730X,0,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,http://images.amazon.com/images/P/002542730X.0...,82


In [22]:
# shape of the data
final_rating.shape

(488742, 9)

In [23]:
# selecting the books which have more than 50 ratings
final_rating = final_rating[final_rating['number_of_ratings'] >= 50]
final_rating.sample(10)

,user_id,isbn,rating,title,author,year,publisher,image_url,number_of_ratings
994,88733,014100018X,0,Chocolat,Joanne Harris,2000,Penguin Books,http://images.amazon.com/images/P/014100018X.0...,104
92099,32773,0451203895,0,4 Blondes,Candace Bushnell,2002,Signet Book,http://images.amazon.com/images/P/0451203895.0...,72
136318,253821,0345353145,0,Sphere,MICHAEL CRICHTON,1988,Ballantine Books,http://images.amazon.com/images/P/0345353145.0...,117
70606,226879,0345337662,10,Interview with the Vampire,Anne Rice,1993,Ballantine Books,http://images.amazon.com/images/P/0345337662.0...,177
5763,264082,0425155943,0,Contagion,Robin Cook,1998,Berkley Publishing Group,http://images.amazon.com/images/P/0425155943.0...,56
7755,91203,0440241073,6,The Summons,John Grisham,2002,Dell Publishing Company,http://images.amazon.com/images/P/0440241073.0...,171
10335,236340,0553280341,5,B Is for Burglar (Kinsey Millhone Mysteries (P...,Sue Grafton,1986,Bantam,http://images.amazon.com/images/P/0553280341.0...,132
82225,68555,0452283205,0,Falling Angels,Tracy Chevalier,2002,Plume Books,http://images.amazon.com/images/P/0452283205.0...,62
114349,245963,0440200563,0,Fine Things,Danielle Steel,1988,Dell,http://images.amazon.com/images/P/0440200563.0...,57
110286,31315,0679433740,0,Paradise,Toni Morrison,1998,Alfred A. Knopf,http://images.amazon.com/images/P/0679433740.0...,73


In [24]:
# shape of the data
final_rating.shape

(62149, 9)

In [25]:
# let's drop the duplicate values
final_rating.drop_duplicates(['user_id', 'title'], inplace=True)
final_rating.shape

(60141, 9)

you can observe that there are some duplicate values in the data


In [26]:
# creating a pivot table
book_pivot = final_rating.pivot(index='title', columns='user_id', values='rating').fillna(0)
book_pivot.head()


user_id,254,2276,2766,2977,3363,3757,4017,4385,6242,6251,...,274004,274061,274301,274308,274808,275970,277427,277478,277639,278418
title,,,,,,,,,,,,,,,,,,,,,
1984,9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1st to Die: A Novel,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2nd Chance,0.0,10.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4 Blondes,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
84 Charing Cross Road,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,10.0,0.0,0.0,0.0,0.0


In [27]:
# finding the shape of the pivot table
book_pivot.shape

(746, 894)

In [28]:
# creating a sparse matrix
from scipy.sparse import csr_matrix
book_matrix = csr_matrix(book_pivot.values)
book_matrix

<746x894 sparse matrix of type '<class 'numpy.float64'>'
	with 15040 stored elements in Compressed Sparse Row format>

In [29]:
# model implementation
from sklearn.neighbors import NearestNeighbors
model_knn = NearestNeighbors(algorithm = 'brute')

In [30]:
# fitting the model
model_knn.fit(book_matrix)

NearestNeighbors(algorithm='brute')

In [31]:
distance,suggestion = model_knn.kneighbors(book_pivot.iloc[1,:].values.reshape(1, -1), n_neighbors = 6)

In [32]:
distance

array([[ 0.        , 53.96295025, 54.01851534, 55.05451843, 55.1724569 ,
        55.21775077]])

In [33]:
suggestion

array([[  1, 184, 539, 535, 372, 654]])

In [34]:
# find the name of the book which is suggested
for i in range(len(suggestion)):
    print(book_pivot.index[suggestion[i]])

Index(['1st to Die: A Novel', 'Exclusive', 'The Cradle Will Fall',
       'The Clinic (Alex Delaware Novels (Paperback))', 'No Safe Place',
       'The Sum of All Fears (Jack Ryan Novels)'],
      dtype='object', name='title')


In [35]:
# storing all the book names in a list
book_names = list(book_pivot.index)

In [36]:
book_names

['1984',
 '1st to Die: A Novel',
 '2nd Chance',
 '4 Blondes',
 '84 Charing Cross Road',
 'A Bend in the Road',
 'A Case of Need',
 'A Child Called \\It\\": One Child\'s Courage to Survive"',
 'A Civil Action',
 'A Cry In The Night',
 'A Darkness More Than Night',
 'A Day Late and a Dollar Short',
 'A Fine Balance',
 'A Great Deliverance',
 'A Heartbreaking Work of Staggering Genius',
 'A Is for Alibi (Kinsey Millhone Mysteries (Paperback))',
 'A Lesson Before Dying (Vintage Contemporaries (Paperback))',
 'A Man Named Dave: A Story of Triumph and Forgiveness',
 'A Man in Full',
 'A Map of the World',
 'A Painted House',
 'A Patchwork Planet',
 'A Prayer for Owen Meany',
 'A Thin Dark Line (Mysteries &amp; Horror)',
 "A Thousand Acres (Ballantine Reader's Circle)",
 'A Time to Kill',
 "A Virtuous Woman (Oprah's Book Club (Paperback))",
 'A Walk to Remember',
 'A Widow for One Year',
 'A Wrinkle In Time',
 'A Wrinkle in Time',
 'A Year in Provence',
 "ANGELA'S ASHES",
 'Abduction',
 'Abou

In [37]:
# importing pickle
import pickle
pickle.dump(model_knn, open('artifacts/model_knn.pkl', 'wb'))
pickle.dump(book_names, open('artifacts/book_names.pkl', 'wb'))
pickle.dump(book_pivot, open('artifacts/book_pivot.pkl', 'wb'))
pickle.dump(final_rating, open('artifacts/final_rating.pkl', 'wb'))

In [38]:
# let's test the model
def recommend(book_name):
    book_id = np.where(book_pivot.index == book_name)[0][0]
    distance,suggestion = model_knn.kneighbors(book_pivot.iloc[book_id,:].values.reshape(1, -1), n_neighbors = 6)
    for i in range(len(suggestion)):
        print(book_pivot.index[suggestion[i]])

In [39]:
# let's test the model
recommend('The Lovely Bones: A Novel')


Index(['The Lovely Bones: A Novel', 'The Bluest Eye', 'Granny Dan',
       'Nights in Rodanthe', 'Drowning Ruth (Oprah's Book Club)', 'Whispers'],
      dtype='object', name='title')
